In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [5]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [9]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [11]:
df=df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [13]:
df.head(5)

,artist,song,text
0,Enya,Christmas Secrets,"Underneath a winter sky, \r\nA distant train ..."
1,Richard Marx,Superstar,You're the queen of your own little \r\nWorld...
2,Lloyd Cole,No Blue Skies,You want to leave me baby be my guest \r\nAll...
3,Bob Seger,Death Row,"Here I am, oh my time's rolling slowly by \r\..."
4,Jason Mraz,Please Don't Tell Her,I hear she's kickin' ass across the board \r\...


In [15]:
df['text'][0]

"Underneath a winter sky,  \r\nA distant train sings out the miles.  \r\nAnd so I wonder can it be,  \r\nWill every mile bring you to me?  \r\n  \r\nA promise made may still come true,  \r\nSo I am waiting here for you.  \r\nIf you don't come, what will I do?  \r\nWho shall I tell my secrets to?  \r\n  \r\nChristmas bells ring out their chimes,  \r\nI hear them echo through the night.  \r\nAnd moonlight shines upon the road,  \r\nAnd trembles on the falling snow.  \r\n  \r\nI look into the midnight blue,  \r\nSo many stars I never knew.  \r\nIf you don't come, what will I do?  \r\nWho shall I tell my secrets to?  \r\n  \r\nI look into the midnight blue,  \r\nSo many stars I never knew.  \r\nIf you don't come, what will I do?  \r\nWho shall I tell my secrets to?\r\n\r\n"

In [17]:
df.shape

(5000, 3)

In [19]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [21]:
import nltk

from nltk.stem.porter import PorterStemmer

In [23]:
stemmer = PorterStemmer()

In [25]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [27]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [33]:
similarity[0]

array([1.        , 0.01063529, 0.04882168, ..., 0.00749474, 0.0020274 ,
       0.02223262])

In [39]:
df[df['song'].str.strip() == "Please Don't Tell Her"]


,artist,song,text
4,Jason Mraz,Please Don't Tell Her,i hear she 's kickin ' ass across the board an...


In [45]:
def recommendation(song_df):
    song_df = song_df.strip().lower()  # Normalize the song name
    idx = df[df['song'].str.strip().str.lower() == song_df].index

    # If no matching song is found
    if len(idx) == 0:
        return f"Song '{song_df}' not found in the database."

    idx = idx[0]  # Get the first match (index)
    
    # Sort songs by similarity score (excluding the song itself)
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

    # Create a list to store recommended songs
    songs = []
    for m_id in distances[1:6]:  # Get the top 5 recommended songs
        songs.append(df.iloc[m_id[0]].song)

    return songs


In [47]:
recommended_songs = recommendation("Please Don't Tell Her")
print(recommended_songs)


["Daddy's Eyes", 'Luv The Hurt Away', 'Are We All We Are', "When She Don't Need Me", 'I Love You']


In [49]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))